# Example: Multi-Agent System (MAS) Simulation

This notebook demonstrates a true multi-agent, event-driven simulation. All components are fully decoupled and communicate only via a `MessageBus`.

The system consists of a `Reservoir` and a `Gate`. A `DigitalTwinAgent` publishes the reservoir's state, and a `LocalControlAgent` subscribes to this information. The control agent uses a PID controller to adjust the gate's opening to maintain a constant water level in the reservoir.

## System Components

### Physical
- **Reservoir (`reservoir_1`)**: The water source.
- **Gate (`gate_1`)**: The outflow, controlled by an agent.

### Agents
- **DigitalTwinAgent**: Publishes the `water_level` of the reservoir to the message bus.
- **LocalControlAgent**: Listens for `water_level` updates and sends `opening` commands to the gate to drive the level towards a setpoint of 12.0m.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.core_engine.testing.simulation_harness import SimulationHarness

# Simulation Setup
simulation_config = {'duration': 300, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

# Communication Topics
RESERVOIR_STATE_TOPIC = "state.reservoir.level"
GATE_ACTION_TOPIC = "action.gate.opening"

# Physical Components
gate_params = {'max_rate_of_change': 0.1, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 1.0}
reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 21e6, 'water_level': 14.0}, parameters={'surface_area': 1.5e6})
gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

# Agent Components
twin_agent = DigitalTwinAgent(agent_id="twin_agent_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
pid_controller = PIDController(Kp=-0.5, Ki=-0.01, Kd=-0.1, setpoint=12.0, min_output=0.0, max_output=gate_params['max_opening'])
control_agent = LocalControlAgent(agent_id="control_agent_gate_1", controller=pid_controller, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=harness.dt)

# Add all components to the harness
harness.add_component(reservoir)
harness.add_component(gate)
harness.add_agent(twin_agent)
harness.add_agent(control_agent)

harness.add_connection("reservoir_1", "gate_1")

# Build and run the simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("MAS simulation complete.")

## Results and Visualization

The plots below show the classic behavior of a feedback control system. The controller adjusts the gate opening to drive the reservoir's water level towards the desired setpoint.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
reservoir_levels = [h['reservoir_1']['water_level'] for h in harness.history]
gate_openings = [h['gate_1']['opening'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Reservoir Level
ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level')
ax1.axhline(y=12.0, color='r', linestyle='--', label='Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('MAS Feedback Control Simulation')
ax1.grid(True)
ax1.legend()

# Gate Opening
ax2.plot(df['Time'], df['Gate Opening'], label='Gate Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()